# Task 2b: Abstractive Text Summarization

In this notebook, you manage challenges arising in large document summarization - input text can exceed model context lengths, generate hallucinated outputs, or trigger out-of-memory errors.

To mitigate these issues, this notebook demonstrates an architecture using prompt chunking and chaining with the [LangChain](https://python.langchain.com/docs/get_started/introduction.html) framework, a toolkit enabling applications leveraging language models.

You explore an approach addressing scenarios when user documents surpass token limits. Chunking splits documents into segments under context length thresholds before sequentially feeding them to models. This chains prompts across chunks, retaining prior context. You apply this approach to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

## Task 2b.1: Environment setup

In this task, you set up your environment and create a Bedrock client that automatically detects your AWS region.

In [19]:
#Create a service client by name using the default session.
import json
import os
import sys
import time
import random
from typing import Any, List, Mapping, Optional

# AWS and Bedrock imports
import boto3

# Get the region programmatically
session = boto3.session.Session()
region = session.region_name or "us-east-1"  # Default to us-east-1 if region not set

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=region)


## Task 2b.2: Summarize long text 

### Configuring LangChain with Boto3

In this task, you specify the LLM for the LangChain Bedrock class, and pass arguments for inference.

In [20]:
# LangChain imports
from langchain_aws import BedrockLLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

# Base LLM configuration
modelId = "amazon.nova-lite-v1:0"

class NovaLiteWrapper(LLM):
    """Wrapper for Nova Lite model that formats inputs correctly."""
    
    @property
    def _llm_type(self) -> str:
        return "nova-lite-wrapper"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Format prompt for Nova Lite and process."""
        # Format the prompt for Nova Lite's expected message structure
        formatted_input = {
            "messages": [
                {
                    "role": "user",
                    "content": [{"text": prompt}]  # Content must be an array with text objects
                }
            ],
            "inferenceConfig": {
                "maxTokens": 2048,
                "temperature": 0,
                "topP": 0.9
            }
        }
        
        # Call Bedrock directly with the properly formatted input
        response = bedrock_client.invoke_model(
            modelId=modelId,
            body=json.dumps(formatted_input)
        )
        
        # Parse the response - updated to handle Nova Lite's response format
        response_body = json.loads(response['body'].read().decode('utf-8'))
        
        # Extract the text from the response
        if 'output' in response_body and 'message' in response_body['output']:
            message = response_body['output']['message']
            if 'content' in message and isinstance(message['content'], list):
                # Extract text from each content item
                texts = []
                for content_item in message['content']:
                    if isinstance(content_item, dict) and 'text' in content_item:
                        texts.append(content_item['text'])
                return ' '.join(texts)
        
        # Fallback if the response format is different
        return str(response_body)
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model_id": modelId}
    
    def get_num_tokens(self, text: str) -> int:
        """Estimate token count - Nova Lite uses roughly 1 token per 4 characters."""
        return len(text) // 4  # Rough approximation

# Create the Nova Lite wrapper
llm = NovaLiteWrapper()


## Creating a Resource-Optimized LLM Wrapper

To handle Bedrock service quotas effectively, we'll create a wrapper class that optimizes resource usage and implements exponential backoff with jitter for API calls.

In [21]:
# Enhanced resource-optimized LLM wrapper with exponential backoff
class ResourceOptimizedLLM(LLM):
    """Wrapper that optimizes resource usage for LLM processing."""
    
    llm: Any  # The base LLM to wrap
    min_pause: float = 30.0  # Minimum pause between requests
    max_pause: float = 60.0  # Maximum pause after throttling
    initial_pause: float = 10.0  # Initial pause between requests
    
    @property
    def _llm_type(self) -> str:
        return f"optimized-{self.llm._llm_type}"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Process with resource optimization and exponential backoff."""
        # Always pause between requests to optimize resource usage
        time.sleep(self.initial_pause)
        
        # Implement retry with exponential backoff
        max_retries = 10  # More retries for important operations
        base_delay = self.min_pause
        
        for attempt in range(max_retries):
            try:
                print(f"Making API call (attempt {attempt+1}/{max_retries})...")
                return self.llm._call(prompt, stop=stop, run_manager=run_manager, **kwargs)
            
            except Exception as e:
                error_str = str(e)
                
                # Handle different types of service exceptions
                if any(err in error_str for err in ["ThrottlingException", "TooManyRequests", "Rate exceeded"]):
                    if attempt < max_retries - 1:
                        # Calculate backoff with jitter to prevent request clustering
                        jitter = random.random() * 0.5
                        wait_time = min(base_delay * (2 ** attempt) + jitter, self.max_pause)
                        
                        print(f"Service capacity reached. Backing off for {wait_time:.2f} seconds...")
                        time.sleep(wait_time)
                    else:
                        print("Maximum retries reached. Consider reducing batch size or increasing delays.")
                        raise
                else:
                    # For non-capacity errors, don't retry
                    print(f"Non-capacity error: {error_str}")
                    raise
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {**self.llm._identifying_params, "initial_pause": self.initial_pause}
    
    def get_num_tokens(self, text: str) -> int:
        """Pass through token counting to the base model."""
        return self.llm.get_num_tokens(text)

# Create the resource-optimized LLM
resource_optimized_llm = ResourceOptimizedLLM(llm=llm, initial_pause=10.0)



<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** This wrapper adds important features for production use:

- Automatic pausing between requests to respect service quotas
- Exponential backoff with jitter for handling throttling exceptions
- Comprehensive error handling and reporting

## Task 2b.3: Loading a text file with many tokens

In this task, you use a copy of [Amazon's CEO letter to shareholders in 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) in the letters directory. You create a function to load the text file and handle potential errors.

In [22]:
# Document loading function
def load_document(file_path):
    """Load document from file."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
        return content
    except Exception as e:
        print(f"Error loading document: {e}")
        return None

# Example usage
shareholder_letter = "../letters/2022-letter.txt"
letter = load_document(shareholder_letter)

if letter:
    num_tokens = resource_optimized_llm.get_num_tokens(letter)
    print(f"Document loaded successfully with {num_tokens} tokens")


Document loaded successfully with 8109 tokens


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** You can safely ignore the warnings and proceed to next cell. We'll address this by chunking the document in the next step.

## Task 2b.4: Splitting the long text into chunks

In this task, you split the text into smaller chunks because it is too long to fit in the prompt. `RecursiveCharacterTextSplitter` in LangChain supports splitting long text into chunks recursively until the size of each chunk becomes smaller than chunk_size.

In [23]:
# Document chunking with conservative settings
def chunk_document(text, chunk_size=4000, chunk_overlap=200):
    """Split document into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " "],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    chunks = text_splitter.create_documents([text])
    print(f"Document split into {len(chunks)} chunks")
    return chunks

# Split the document into chunks
if letter:
    docs = chunk_document(letter, chunk_size=4000, chunk_overlap=200)
    
    if docs:
        num_docs = len(docs)
        num_tokens_first_doc = resource_optimized_llm.get_num_tokens(docs[0].page_content)
        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")


Document split into 10 chunks
Now we have 10 documents and the first one has 543 tokens


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The `chunk_size` parameter controls how large each chunk will be. Larger chunks provide more context but require more processing resources. The `chunk_overlap` parameter ensures some continuity between chunks.

## Task 2b.5: Summarizing chunks and combining them

In this task, you implement two approaches for summarizing chunked documents: using LangChain's built-in summarize chain and a custom manual implementation that provides better control over resource usage.

## Understanding Implementation Approaches

This notebook demonstrates two different approaches for summarizing large documents with AWS Bedrock:

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** We've included both a standard LangChain implementation and a custom implementation to show the tradeoffs between convenience and control when building production applications.

### Two Paths to the Same Goal

1. **Standard LangChain Implementation** (`process_documents_with_pacing`):
   - Uses LangChain's built-in summarization chains
   - Requires less code and is easier to implement
   - Abstracts away the underlying complexity
   - Great for rapid prototyping and simple use cases

2. **Custom Refine Implementation** (`manual_refine_with_optimization`):
   - Builds the refinement process step-by-step
   - Provides complete visibility into prompts and processing
   - Offers granular error handling for each document chunk
   - Allows precise control over API call timing and retry logic

While both achieve the same end result, the custom implementation gives you more control over the entire process, which is crucial when working with service quotas and building production-ready applications.

In real-world scenarios, you might start with the standard implementation during development and then move to a custom implementation when you need more control over resource usage, error handling, or prompt engineering.

### Standard LangChain Implementation

In [24]:
# Custom document processing with controlled pacing
def process_documents_with_pacing(docs, chain_type="refine", verbose=True):
    """Process documents with pacing to optimize resource usage."""
    
    # Configure the chain
    summary_chain = load_summarize_chain(
        llm=resource_optimized_llm,
        chain_type=chain_type,  # "refine" processes sequentially, good for resource optimization
        verbose=verbose
    )
    
    # Process with additional error handling
    try:
        result = summary_chain.invoke(docs)
        return result
    except ValueError as error:
        if "AccessDeniedException" in str(error):
            print(f"\n\033[91mAccess Denied: {error}\033[0m")
            print("\nTo troubleshoot this issue, please check:")
            print("1. Your IAM permissions for Bedrock")
            print("2. Model access permissions")
            print("3. AWS credentials configuration")
            return {"output_text": "Error: Access denied. Check permissions."}
        else:
            print(f"\n\033[91mError during processing: {error}\033[0m")
            return {"output_text": f"Error during processing: {str(error)}"}


### Custom Refine Implementation with Enhanced Resource Optimization

In [25]:
# Manual implementation of resource-optimized processing for refine chain
def manual_refine_with_optimization(docs, llm, verbose=True):
    """Manually implement refine chain with resource optimization."""
    if not docs:
        return {"output_text": "No documents to process."}
    
    # Process first document to get initial summary
    print(f"Processing initial document (1/{len(docs)})...")
    
    # Simple prompt for initial document
    initial_prompt = """Write a concise summary of the following:
    "{text}"
    CONCISE SUMMARY:"""
    
    # Process first document
    try:
        current_summary = llm(initial_prompt.format(text=docs[0].page_content))
        print("Initial summary created successfully.")
    except Exception as e:
        print(f"Error creating initial summary: {e}")
        return {"output_text": "Failed to create initial summary."}
    
    # Process remaining documents with refine approach
    for i, doc in enumerate(docs[1:], start=2):
        print(f"Refining with document {i}/{len(docs)}...")
        
        # Refine prompt
        refine_prompt = """Your job is to refine an existing summary.
        We have an existing summary: {existing_summary}
        
        We have a new document to add information from: {text}
        
        Please update the summary to incorporate new information from the document.
        If the document doesn't contain relevant information, return the existing summary.
        
        REFINED SUMMARY:"""
        
        try:
            # Apply resource optimization between requests
            time.sleep(10.0)  # Base delay between requests
            
            # Update the summary
            current_summary = llm(refine_prompt.format(
                existing_summary=current_summary,
                text=doc.page_content
            ))
            
            if verbose:
                print(f"Successfully refined with document {i}")
        except Exception as e:
            print(f"Error during refinement with document {i}: {e}")
            # Apply exponential backoff
            backoff = min(10.0 * (2 ** (i % 5)) + (random.random() * 2), 30)
            print(f"Backing off for {backoff:.2f} seconds...")
            time.sleep(backoff)
            
            # Try one more time
            try:
                current_summary = llm(refine_prompt.format(
                    existing_summary=current_summary,
                    text=doc.page_content
                ))
            except Exception as retry_error:
                print(f"Retry failed for document {i}: {retry_error}")
                # Continue with current summary rather than failing completely
    
    return {"output_text": current_summary}


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The manual implementation gives you more control over:

- The exact prompts used for summarization
- Error handling and recovery
- Resource optimization between API calls
- Graceful degradation when errors occur

## Task 2b.6: Main Execution Function

Now we'll create a main function that orchestrates the entire document summarization process.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The main function (`summarize_document`) allows you to choose which implementation to use based on the `chain_type` parameter, making it easy to compare results and performance.

In [26]:
# Main execution function
def summarize_document(file_path, chunk_size=4000, chain_type="refine"):
    """Main function to summarize a document."""
    
    print(f"Starting document summarization process for: {file_path}")
    
    # Load the document
    document_text = load_document(file_path)
    if not document_text:
        return "Failed to load document."
    
    print(f"Document loaded successfully. Length: {len(document_text)} characters")
    
    # Split into chunks
    docs = chunk_document(document_text, chunk_size=chunk_size, chunk_overlap=200)
    
    # If document is very large, provide a warning
    if len(docs) > 15:
        print(f"Warning: Document is large ({len(docs)} chunks). Processing may take some time.")
        
        # For very large documents, consider using a subset for testing
        if len(docs) > 30:
            print("Document is extremely large. Consider using a smaller chunk_size or processing a subset.")
            # Optional: process only a subset for testing
            # docs = docs[:15]
    
    # Process the documents
    print(f"Processing document using '{chain_type}' chain type...")
    
    # Use the appropriate processing method based on chain type
    if chain_type == "refine":
        # Use our manual implementation for better control over resource optimization
        result = manual_refine_with_optimization(docs, resource_optimized_llm)
    else:
        # Use standard LangChain implementation for other chain types
        result = process_documents_with_pacing(docs, chain_type=chain_type)
    
    # Return the result
    if result and "output_text" in result:
        print("\nSummarization completed successfully!")
        return result["output_text"]
    else:
        print("\nSummarization failed or returned no result.")
        return "Summarization process did not produce a valid result."


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** Depending on your number of documents, Bedrock request rate quota, and configured retry settings - the summarization process may take some time to run.

## Task 2b.7: Run the Summarization

Let's run the summarization on the shareholder letter. By default, the summarize_document() function uses the refine chain. To enable map_reduce: 

- Comment out the following line: `summary = summarize_document(document_path, chunk_size=4000, chain_type="refine")`
- Uncomment the following line: `# summary = summarize_document(document_path, chunk_size=4000, chain_type="map_reduce")`


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** Don't worry if you notice error messages during execution. Your code includes robust error handling that will automatically retry failed requests with exponential backoff. This is normal behavior when working with service quotas and demonstrates how production-ready applications should handle API limitations.

In [27]:
# Example usage
if __name__ == "__main__":
    # Path to your document
    document_path = "../letters/2022-letter.txt"
    
    # Summarize with different options
    # Option 1: Standard refine chain (sequential processing, good for resource optimization)
    summary = summarize_document(document_path, chunk_size=4000, chain_type="refine")
    
    # Option 2: For comparison, you could try map_reduce (but be careful with service quotas)
    # summary = summarize_document(document_path, chunk_size=4000, chain_type="map_reduce")
    
    # Print the final summary
    print("\n=== FINAL SUMMARY ===\n")
    print(summary)


Starting document summarization process for: ../letters/2022-letter.txt
Document loaded successfully. Length: 32438 characters
Document split into 10 chunks
Processing document using 'refine' chain type...
Processing initial document (1/10)...
Making API call (attempt 1/10)...
Initial summary created successfully.
Refining with document 2/10...
Making API call (attempt 1/10)...
Successfully refined with document 2
Refining with document 3/10...
Making API call (attempt 1/10)...
Successfully refined with document 3
Refining with document 4/10...
Making API call (attempt 1/10)...
Successfully refined with document 4
Refining with document 5/10...
Making API call (attempt 1/10)...
Successfully refined with document 5
Refining with document 6/10...
Making API call (attempt 1/10)...
Successfully refined with document 6
Refining with document 7/10...
Making API call (attempt 1/10)...
Successfully refined with document 7
Refining with document 8/10...
Making API call (attempt 1/10)...
Success

You have now experimented with using prompt chunking and chaining with the LangChain framework to summarize large documents while mitigating issues arising from long input text.

## Understanding the Key Components

Let's review the key components of our solution:

1. **Resource Optimization**: The `ResourceOptimizedLLM` wrapper manages API calls to stay within Bedrock service quotas by:
   - Adding pauses between requests (controlled by `initial_pause`)
   - Implementing exponential backoff with jitter when throttling occurs
   - Providing comprehensive error handling and recovery

2. **Document Chunking**: The `chunk_document` function splits large documents into manageable pieces:
   - `chunk_size` controls the maximum size of each chunk (4000 characters)
   - `chunk_overlap` ensures context continuity between chunks (200 characters)
   - Natural text separators (`\n\n`, `\n`, `.`, etc.) are used to avoid breaking mid-paragraph

3. **Summarization Approaches**:
   - **Refine Chain**: Processes chunks sequentially, refining the summary with each new chunk
   - **Map-Reduce**: Summarizes each chunk independently, then combines and summarizes those summaries

4. **Error Handling**: Comprehensive error handling ensures the process can recover from:
   - Service throttling and capacity limits
   - Access permission issues
   - Other API errors

## Try it yourself

- Change the prompts to your specific usecase and evaluate the output of different models.
- Experiment with different chunk sizes to find the optimal balance between context preservation and processing efficiency.
- Try different summarization chain types (`refine` vs `map_reduce`) and compare the results.
- Adjust the resource optimization parameters based on your Bedrock quota limits.

### Practical Applications

This approach can be applied to summarize various types of long-form content:
- Customer service call transcripts
- Meeting transcripts and notes
- Research papers and technical documents
- Legal documents and contracts
- Books, articles, and blog posts

### Best Practices

When implementing this solution in production:

1. **Monitor API usage**: Keep track of your API calls to stay within quota limits
2. **Optimize chunk size**: Balance between context preservation and processing efficiency
3. **Implement proper error handling**: Ensure your application can gracefully handle API errors
4. **Consider caching**: Cache results to avoid redundant API calls for frequently accessed documents
5. **Test with various document types**: Different content may require different chunking strategies

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file and continue with **Task 3**.